**Análisis y trabajo con el dataset 20NewsGroups**

<p align="justify">En esta parte del proyecto vamos a trabajar con el dataset 20NewsGroups que engloba mensajes de 20 clases distintas de mensajes de noticias.

# Análisis del dataset y preprocesado

<p align="justify">Primero vamos a realizar una prueba para analizar el dataset, para ello vamos a empezar cargando un subconjunto de 7 clases en lugar de las 20 originales.

<p align="justify">Sobre este subconjunto, vamos a probar a realizar una clasificación sencilla usando un clasificador bayesinano y vamos a enseñar las 20 palabras más importantes a la hora de realizar esta clasificación en el proceso de entrenamiento.

**Codificación del texto**
<p align="justify">Una parte fundamental del procesamiento de lenguaje natural es la codificación del texto, en nuestro caso vamos a usar TF-IDF (Term Frequency-Inverse Document Frequency). Esta técnica de codificación usa la frecuencia del término (TF) y la inversa de la frecuencia en el documento (Inverse Document Frequency).

<p align="justify">Con esta técnica podemos medir la importancia de un término dentro una clase y entre todas, ya que una palabra que aparece repetidamente en una clase será más relevante que una que aparezca muchas veces en todas las clases.

<p align="justify">Podemos definir la fórmula de TD-IDF como la siguiente:
TF-IDF(t,d,D) = TF(t,d) * IDF(t,D).

<p align="justify">Donde TF(t,d) es la frecuencia de un término t en un documento t. IDF(t,D) es la frecuencia inversa de un término t en el conjunto de documentos D. Para calcular IDF(t,D) usamos log(N/n_t) siendo N el número total de documentos que contiene D y n_t el número de documentos que contienen el término t.



In [ ]:
!pip install numpy
!pip install scikit-learn
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\34653\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import numpy as np
# Función auxiliar para ver las n palabras más importantes a la hora de clasificar
def print_n_most_relevant_words(n, classifier, vectorizer, data):
  probabilities = np.exp(classifier.feature_log_prob_)
  probabilities = np.argsort(-probabilities)
  feature_names = vectorizer.get_feature_names_out()
  for i, target_name in enumerate(data.target_names):
    n_words_idx = probabilities[i][:n]
    words = [feature_names[idx] for idx in n_words_idx]
    print('\n' + target_name + ': ' + ' '.join(words))


In [ ]:
# Importamos el dataset, el vectorizador y el clasificador
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
# Elegimos las 8 categorías que vamos a cargar
categories = [
  'alt.atheism',
  'comp.graphics',
  'comp.os.ms-windows.misc',
  'comp.sys.ibm.pc.hardware',
  'comp.sys.mac.hardware',
  'comp.windows.x',
  'sci.crypt',
  'sci.electronics']
# Cargamos el dataset
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
# Vectorizamos el texto
vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(newsgroups_train.data)
vectors_test = vectorizer.transform(newsgroups_test.data)
clf = MultinomialNB()
clf.fit(vectors_train, newsgroups_train.target)
print_n_most_relevant_words(20, clf, vectorizer, newsgroups_train)
# Realizamos la predicción con los datos de test
pred = clf.predict(vectors_test)
print('\n' + classification_report(newsgroups_test.target, pred, target_names=newsgroups_test.target_names))



alt.atheism: the of to that is in you and it edu god keith not are be he caltech was as we

comp.graphics: the to of and graphics is in for it edu from that you on this image have be or university

comp.os.ms-windows.misc: the windows to it edu and is of in for you that file have dos with from on driver this

comp.sys.ibm.pc.hardware: the to and scsi it of is drive for edu with in ide on that card com have bus my

comp.sys.mac.hardware: the to edu of and mac is apple it in for that with on you from have my this drive

comp.windows.x: the to window of and is in com motif it for edu this mit on server that you from with

sci.crypt: the to of and key is that clipper in it encryption be chip com they this for you as government

sci.electronics: the to of and is in edu you it for that on are com from have be this with if

                          precision    recall  f1-score   support

             alt.atheism       0.99      0.91      0.95       319
           comp.graphics       0.82  


<p align="justify">Como se puede apreciar en la ejecución de la celda de código anterior, entras las palabras más relevantes a la hora de clasificar, hay palabras muy genéricas como pueden ser artículos (the), preposiciones (to, of, in, from, on, with), demostrativos (this, that), pronombres personales(You, We), conjunciones (and). También nos encontramos palabras como edu que dan información no sobre la temático del texto, sino del remitente (haciendo referencia a su pertenencia a una universidad).

<p align="justify">El resultado anterior justifica realizar un proprocesado a nuestros datos, para ello vamos a eliminar las cabeceras, los pies de página y las citas. También vamos a eliminar las palabras que no aportan información que hemos citado anteriormente.

<p align="justify">Otro preprocesado que vamos a realizar sobre el dataset es un generalizado de palabras, así, podremos reducir el tamaño del vocabulario propio de cada clase teniendo en cuenta la palabra base. Un ejemplo de esto sería de la palabra 'having' obtener la palabra 'have'.

<p align="justify">Aplicando este preprocesadolos resultados que obtengamos serán más cercanos a la realidad, los actuales tienen ruido debido a la presencia de estas palabras.

In [ ]:
# Función auxiliar para realizar la generalización
def lemmatize(tokens, lemmatizer):
    lemmatized_tokens = []
    for token in tokens:
        lemmatized_tokens.append(lemmatizer.lemmatize(token))
    return lemmatized_tokens

In [ ]:
# Importamos el lemmatizer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
# Cargamos una lista de signos de puntuación para eliminarlos al preprocesar
punctuations = string.punctuation + '...' + '--' + '``' + '\'' + ''
# Función para realizar el tokenizado desde el texto (usa la función auxiliar de stemming)
def tokenize(text, lemmatizer=WordNetLemmatizer()):
    tokens = nltk.word_tokenize(text)
    tokens = [i for i in tokens if i not in punctuations]
    lemmatized_tokens = lemmatize(tokens, lemmatizer)
    #stemmed_tokens = [i for i in stemmed_tokens if i not in punctuations]
    return lemmatized_tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\34653\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\34653\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# Cargamos el dataset sin cabeceras, pies de página ni citas
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'),categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),categories=categories)
# Tokenizamos y clasificamos el texto
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
vectors_train = vectorizer.fit_transform(newsgroups_train.data)
vectors_test = vectorizer.transform(newsgroups_test.data)
clf = MultinomialNB()
clf.fit(vectors_train, newsgroups_train.target)
print_n_most_relevant_words(20, clf, vectorizer, newsgroups_train)
# Realizamos la predicción con los datos de test
pred = clf.predict(vectors_test)
print('\n' + classification_report(newsgroups_test.target, pred, target_names=newsgroups_test.target_names))

C:\Users\34653\anaconda3\envs\colab\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\34653\anaconda3\envs\colab\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(



alt.atheism: '' n't god wa 's people think religion atheist say just doe thing belief moral bible did argument ha said

comp.graphics: file image graphic program thanks n't know format '' 's doe color looking need point 'm wa 3d help package

comp.os.ms-windows.misc: window file driver problem '' program 's n't use font 3.1 card know thanks 'ax doe wa using version like

comp.sys.ibm.pc.hardware: drive card controller ide n't scsi bus disk monitor ha problem doe thanks 's pc '' port know board wa

comp.sys.mac.hardware: mac apple drive problem 's n't doe monitor card '' ha know thanks simms work just quadra use 'm centris

comp.windows.x: x window server widget '' motif application thanks 's use program file n't doe display problem using ha know client

sci.crypt: key '' chip encryption n't clipper government 's phone people nsa ha wa just use escrow algorithm doe security law

sci.electronics: 's n't wa '' circuit power like know use doe line ground good chip used voltage ha current 

<p align="justify">Podemos observar como los resultados han empeorado con respecto a la clasificación realizada sin preprocesar. Pero, si nos fijamos en las palabras más significativas a la hora de clasificar, ahora sí que podemos ver como estas se ajustan a la temática de la que se está hablando, por lo que podemos concluir que los resultados actuales reflejan mejor la realidad que los anteriores.

# Comparación de clasificadores

<p align="justify"> Una vez hemos realizado el análisis del dataset y hemos justificado porqué es necesario realizar un preprocesamiento de los datos, procedemos a realizar una comparativa de distintos clasificadores para valorar los rendimientos obtenidos. Para ello utilizaremos el dataset completo de 20 categorías. Para realizar una comparación adecuado usaremos validación cruzada estratificada para así mantener el porcentaje de presencia de cada clase en los distintos conjuntos que se crean a la hora de realizar la validación.

In [ ]:
# Creamos una función para imprimir por pantalla el informe de clasificación medio de todos los informes al usar Kfold CrossValidation
def print_mean_classification_report(classification_reports):
  n_folds = len(classification_reports)
  total_report = {}

  # Sumamos para cada métrica
  for report in classification_reports:
      for metric, scores in report.items():

        if metric not in total_report:
          if metric != 'accuracy':
            total_report[metric] = scores

        else:
          if metric not in ['accuracy']:
            total_report[metric] = {k: total_report[metric][k] + scores[k] for k in scores}

  # Dividimos entre el número de folds de validación
  mean_report = {metric: {k: v/n_folds for k, v in scores.items()} for metric, scores in total_report.items()}

  # Damos formato
  fmt_string = '{:<25} {:<25} {:<25} {:<25} {:<25}'

  # Mostramos por pantalla las métricas
  print(fmt_string.format('', 'precision', 'recall', 'f1-score', 'support'))

  # Mostramos el informe de clasificiación
  for metric, scores in mean_report.items():
      precision = scores['precision']
      recall = scores['recall']
      f1_score = scores['f1-score']
      support = scores['support']
      print(fmt_string.format(metric, precision, recall, f1_score, support))

**Clasificador de Bayes Ingenuo**

<p align="justify">El primer clasificador que vamos a usar es el que hemos estado trabajando para presentar el dataset y la justificación del preprocesado. Se trata del clasificador de Bayes ingenuo. Es un clasificador probabilístico cuyo fundamento recae en la regla de Bayes sobre la probabilidad condicionada. Recibe el adjetivo de ingenuo ya que el modelo asume que la independencia de las características usadas en la clasificación.

In [ ]:
import warnings

# Ignoramos los warnings de la categoría UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Importamos Validación Cruzada Estratificada
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

# Nos quedamos con los nombres de las clases para el informe de clasificación
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
labels_name = dataset.target_names

# Cargamos el dataset completo
data, labels = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), return_X_y=True)
data, labels = np.asarray(data), np.asarray(labels)

# Creamos una lista para guardar los distintos classification reports y las accuracies
classification_reports = []
accuracies = []

# Obtenemos los conjuntos de entrenamiento y test para cada ejecución
for i, (train_index, test_index) in enumerate(skfold.split(data, labels)):
  data_train, data_test = data[train_index], data[test_index]
  labels_train, labels_test = labels[train_index], labels[test_index]

  # Realizamos el preprocesamiento de cada conjunto
  vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
  vectors_train = vectorizer.fit_transform(data_train)
  vectors_test = vectorizer.transform(data_test)

  # Realizamos el entrenamiento
  clf = MultinomialNB()
  clf.fit(vectors_train, labels_train)

  # Realizamos la predicción con los datos de test
  accuracy = clf.score(vectors_test, labels_test)
  accuracies.append(accuracy)
  pred = clf.predict(vectors_test)
  report = classification_report(labels_test, pred, target_names=labels_name, output_dict=True, zero_division=0)
  classification_reports.append(report)

print_mean_classification_report(classification_reports)
print("Accuracy: " , np.mean(np.array(accuracies)))

                          precision                 recall                    f1-score                  support                  
alt.atheism               0.8219850490011782        0.30802215189873416       0.44591151391435224       79.9                     
comp.graphics             0.7095113692718791        0.6845150431306543        0.6957521545839586        97.3                     
comp.os.ms-windows.misc   0.7374916216458959        0.664904143475572         0.6984956435839692        98.5                     
comp.sys.ibm.pc.hardware  0.646102483377013         0.7688414759843331        0.7017831491701538        98.2                     
comp.sys.mac.hardware     0.8519460339211996        0.7051224226804125        0.7704828578316578        96.3                     
comp.windows.x            0.8144542830147387        0.8431148216862502        0.827806738359415         98.8                     
misc.forsale              0.835585846458452         0.7406164527666736        0.7837513706

**KNN**

<p align="justify">Otro clasificador que vamos a usar es KNN, un algoritmo de clasificación sencillo dónde se utiliza la distancia del dato a clasificar con respecto a los otros datos clasificados, teniendo en cuenta los K vecinos más cercanos. De esos vecinos, se obtiene como predicción la clase a la que pertenezcan la mayor proporción de vecinos.

In [ ]:
import warnings

# Ignoramos los warnings de la categoría UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Importamos el clasificador
from sklearn.neighbors import KNeighborsClassifier

# Importamos Validación Cruzada Estratificada
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

# Nos quedamos con los nombres de las clases para el informe de clasificación
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
labels_name = dataset.target_names

# Cargamos el dataset completo
data, labels = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), return_X_y=True)
data, labels = np.asarray(data), np.asarray(labels)

# Creamos una lista para guardar los distintos classification reports y las accuracies
classification_reports = []
accuracies = []

# Obtenemos los conjuntos de entrenamiento y test para cada ejecución
for i, (train_index, test_index) in enumerate(skfold.split(data, labels)):
  data_train, data_test = data[train_index], data[test_index]
  labels_train, labels_test = labels[train_index], labels[test_index]

  # Realizamos el preprocesamiento de cada conjunto
  vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
  vectors_train = vectorizer.fit_transform(data_train)
  vectors_test = vectorizer.transform(data_test)

  # Creamos el clasificador
  knn = KNeighborsClassifier(n_neighbors=5)

  # Realizamos el entrenamiento
  knn.fit(vectors_train,labels_train)

  # Realizamos la predicción con los datos de test
  accuracy = knn.score(vectors_test, labels_test)
  accuracies.append(accuracy)
  pred = knn.predict(vectors_test)
  report = classification_report(labels_test, pred, target_names=labels_name, output_dict=True, zero_division=0)
  classification_reports.append(report)

print_mean_classification_report(classification_reports)
print("Accuracy: " , np.mean(np.array(accuracies)))

                          precision                 recall                    f1-score                  support                  
alt.atheism               0.8396621580174213        0.13520569620253164       0.23123711712079986       79.9                     
comp.graphics             0.8421158579982109        0.10793183252682517       0.1906077375500038        97.3                     
comp.os.ms-windows.misc   0.5535793549026866        0.299443413729128         0.17967997750795908       98.5                     
comp.sys.ibm.pc.hardware  0.36069361082731793       0.12525252525252525       0.14696881629854897       98.2                     
comp.sys.mac.hardware     0.8066666666666666        0.03217353951890034       0.061670873449890015      96.3                     
comp.windows.x            0.9052777777777777        0.06985157699443414       0.12944579477289758       98.8                     
misc.forsale              0.9214285714285715        0.055343993267410053      0.1037482321

**Regresión Logística Multinomial**

<p align="justify">También vamos a comparar usando la regresión logística multinomial, modelo que calcula la probabilidad de pertenecer a una clase calculando la relación entre una variable dependiente y varias variables independientes. Se utilizan varias funciones logísticas para calcular la probabilidad de pertenecer a cada una de las clases de la variable dependiente, en función de las variables independientes.

Usaremos saga como solver de la regresión logística debido al tamaño del dataset y su dimensionalidad.

In [ ]:
import warnings

# Ignoramos los warnings de la categoría UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Importamos el clasificador
from sklearn.linear_model import LogisticRegression

# Importamos Validación Cruzada Estratificada
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

# Nos quedamos con los nombres de las clases para el informe de clasificación
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
labels_name = dataset.target_names

# Cargamos el dataset completo
data, labels = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), return_X_y=True)
data, labels = np.asarray(data), np.asarray(labels)

# Creamos una lista para guardar los distintos classification reports y las accuracies
classification_reports = []
accuracies = []

# Obtenemos los conjuntos de entrenamiento y test para cada ejecución
for i, (train_index, test_index) in enumerate(skfold.split(data, labels)):
  data_train, data_test = data[train_index], data[test_index]
  labels_train, labels_test = labels[train_index], labels[test_index]

  # Realizamos el preprocesamiento de cada conjunto
  vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
  vectors_train = vectorizer.fit_transform(data_train)
  vectors_test = vectorizer.transform(data_test)

  # Creamos el clasificador
  lr_multinomial = LogisticRegression(multi_class='multinomial', solver='saga')

  # Realizamos el entrenamiento
  lr_multinomial.fit(vectors_train, labels_train)

  # Realizamos la predicción con los datos de test
  accuracy = lr_multinomial.score(vectors_test, labels_test)
  accuracies.append(accuracy)
  pred = lr_multinomial.predict(vectors_test)
  report = classification_report(labels_test, pred, target_names=labels_name, output_dict=True, zero_division=0)
  classification_reports.append(report)

print_mean_classification_report(classification_reports)
print("Accuracy: " , np.mean(np.array(accuracies)))

                          precision                 recall                    f1-score                  support                  
alt.atheism               0.6440589107724657        0.603386075949367         0.6202849922120348        79.9                     
comp.graphics             0.710792593408503         0.7102566799915844        0.7095084979654057        97.3                     
comp.os.ms-windows.misc   0.7056111564633679        0.682106782106782         0.6926004602107583        98.5                     
comp.sys.ibm.pc.hardware  0.7055748783490752        0.701669758812616         0.7030914506426363        98.2                     
comp.sys.mac.hardware     0.8133089290024124        0.7113187285223368        0.7571022139755749        96.3                     
comp.windows.x            0.8128199976692887        0.8006493506493506        0.8062527743507211        98.8                     
misc.forsale              0.7755881559914389        0.7764569745423942        0.7747497404

**SVM**

<p align="justify">Otro clasificador que vamos a usar es SVM (Support Vector Machines) que clasifica obteniendo hiperplanos que se usan para separar los datos en las distintas clases. Vamos a usar como kernel RBF (Radial Basis Function) ya que nos permite tomar decisiones no lineales en espacios de características de alta dimensionalidad como es el dataset 20newsgroups.

In [ ]:
import warnings

# Ignoramos los warnings de la categoría UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Importamos el clasificador
from sklearn.svm import SVC

# Importamos Validación Cruzada Estratificada
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

# Nos quedamos con los nombres de las clases para el informe de clasificación
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
labels_name = dataset.target_names

# Cargamos el dataset completo
data, labels = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), return_X_y=True)
data, labels = np.asarray(data), np.asarray(labels)

# Creamos una lista para guardar los distintos classification reports y las accuracies
classification_reports = []
accuracies = []

# Obtenemos los conjuntos de entrenamiento y test para cada ejecución
for i, (train_index, test_index) in enumerate(skfold.split(data, labels)):
  data_train, data_test = data[train_index], data[test_index]
  labels_train, labels_test = labels[train_index], labels[test_index]

  # Realizamos el preprocesamiento de cada conjunto
  vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
  vectors_train = vectorizer.fit_transform(data_train)
  vectors_test = vectorizer.transform(data_test)

  # Creamos el clasificador
  svm = SVC(kernel='rbf')

  # Realizamos el entrenamiento
  svm.fit(vectors_train, labels_train)

  # Realizamos la predicción con los datos de test
  accuracy = svm.score(vectors_test, labels_test)
  accuracies.append(accuracy)
  pred = svm.predict(vectors_test)
  report = classification_report(labels_test, pred, target_names=labels_name, output_dict=True, zero_division=0)
  classification_reports.append(report)

print_mean_classification_report(classification_reports)
print("Accuracy: " , np.mean(np.array(accuracies)))


                          precision                 recall                    f1-score                  support                  
alt.atheism               0.6302352301937099        0.6346518987341773        0.6299055283722305        79.9                     
comp.graphics             0.6685278069533926        0.7338523038081212        0.6985874382967128        97.3                     
comp.os.ms-windows.misc   0.7278168131088877        0.6728715728715728        0.6983612140998225        98.5                     
comp.sys.ibm.pc.hardware  0.7251617672421224        0.7138631210059782        0.7189145507542806        98.2                     
comp.sys.mac.hardware     0.8407955689341277        0.6905605670103093        0.7570760743700322        96.3                     
comp.windows.x            0.8495400808228372        0.7904968047825192        0.818283219295872         98.8                     
misc.forsale              0.8058419173676115        0.7713023353671364        0.7867051315

**Random Forest**
<p align="justify">Otro clasificador con el que vamos a comparar es RandomForest, algoritmo en el que construímos un número determinado de arboles de decisión, cada uno de ellos se entra con una muestra aleatoria del conjunto de datos de manera independiente y se usa para clasificar un dato. La clasificación que usa el algoritmo es la que hacen la mayoría de los árboles.

In [ ]:
import warnings

# Ignoramos los warnings de la categoría UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Importamos el clasificador
from sklearn.ensemble import RandomForestClassifier

# Importamos Validación Cruzada Estratificada
from sklearn.model_selection import StratifiedKFold
skfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

# Nos quedamos con los nombres de las clases para el informe de clasificación
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
labels_name = dataset.target_names

# Cargamos el dataset completo
data, labels = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), return_X_y=True)
data, labels = np.asarray(data), np.asarray(labels)

# Creamos una lista para guardar los distintos classification reports y las accuracies
classification_reports = []
accuracies = []

# Obtenemos los conjuntos de entrenamiento y test para cada ejecución
for i, (train_index, test_index) in enumerate(skfold.split(data, labels)):
  data_train, data_test = data[train_index], data[test_index]
  labels_train, labels_test = labels[train_index], labels[test_index]

  # Realizamos el preprocesamiento de cada conjunto
  vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
  vectors_train = vectorizer.fit_transform(data_train)
  vectors_test = vectorizer.transform(data_test)

  # Creamos el clasificador
  rf = RandomForestClassifier(n_estimators=100)

  # Realizamos el entrenamiento
  rf.fit(vectors_train, labels_train)

  # Realizamos la predicción con los datos de test
  accuracy = rf.score(vectors_test, labels_test)
  accuracies.append(accuracy)
  pred = rf.predict(vectors_test)
  report = classification_report(labels_test, pred, target_names=labels_name, output_dict=True, zero_division=0)
  classification_reports.append(report)

print_mean_classification_report(classification_reports)
print("Accuracy: " , np.mean(np.array(accuracies)))

                          precision                 recall                    f1-score                  support                  
alt.atheism               0.5553893041282139        0.463243670886076         0.5034981566313669        79.9                     
comp.graphics             0.6115962658175936        0.6002945508100146        0.6047559919388326        97.3                     
comp.os.ms-windows.misc   0.6071755966218941        0.6648938363224077        0.6344507282517873        98.5                     
comp.sys.ibm.pc.hardware  0.6119525934493146        0.6334570191713048        0.6211060475759929        98.2                     
comp.sys.mac.hardware     0.7219801689676302        0.6584192439862543        0.6877660635540344        96.3                     
comp.windows.x            0.7266208234733089        0.7742630385487528        0.7489705516600063        98.8                     
misc.forsale              0.7124313664859883        0.7487586787292237        0.7295190647

<p align="justify"> De los resultados obtenidos podemos observar que tenemos tres algoritmos que realizan una clasificación con una precisión media de entre el 73-75%. Estos son: Clasificador de Bayes Ingenuo, Regresión Logística Multinomial y SVM.



# Conclusión

<p align="justify">En este capítulo hemos hecho una introducción al Procesamiento de Lenguaje Natural, para ello hemos hecho un estudio del dataset 20newsgroups. Hemos analizado el comportamiento del dataset con un subconjunto, y una vez que hemos demostrado la necesidad del preprocesamiento hemos realizado una comparación de distintos métodos de clasificación para evaluar su rendimiento en el estudio de este conjunto de datos. En el siguiente capítulo veremos como aplicar esto al problema propio que vamos a desarrollar en este trabajo.